In [1]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 모듈 모음
import modules as mds

In [8]:
# 누적 설비용량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total = pd.read_excel("dataset/지역별_누적_설비용량.xlsx")
data_2024 = pd.read_excel("dataset/2024년도_신규_설비용량.xlsx")
data_2023 = pd.read_excel("dataset/2023년도_신규_설비용량.xlsx")
data_2022 = pd.read_excel("dataset/2022년도_신규_설비용량.xlsx")
data_2021 = pd.read_excel("dataset/2021년도_신규_설비용량.xlsx")

# 데이터프레임으로 변환
# ic = Installed Capacity = 설비용량
ic_total = pd.DataFrame(data_total)
ic_2024 = pd.DataFrame(data_2024)
ic_2023 = pd.DataFrame(data_2023)
ic_2022 = pd.DataFrame(data_2022)
ic_2021 = pd.DataFrame(data_2021)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(ic_total)
mds.delete_comma(ic_2024)
mds.delete_comma(ic_2023)
mds.delete_comma(ic_2022)
mds.delete_comma(ic_2021)

# 컴럼 통일화를 위해 capacity_total에 "합계" 컬럼 추가
ic_total["합계"] = ic_total.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
ic_total = pd.concat([ic_total, ic_2024, ic_2023, ic_2022, ic_2021], ignore_index=True)

# 각 연도별 누적값 계산
for i in range(1, len(ic_total)):
    ic_total.iloc[i, 1:] = ic_total.iloc[i-1, 1:] - ic_total.iloc[i, 1:]

# 연도 정리
ic_total.rename(columns={"Unnamed: 0": "연도"}, inplace=True)
ic_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 오름차순 정렬 (주석 처리 시 내림차순)
# ic_total.sort_values(by="연도", inplace=True)
# ic_total.reset_index(drop=True, inplace=True)

# 2020~2024 각 연도의 지역별 누적 설비용량
ic_total.head()

ic_total.to_excel("dataset/지역별_연도별_설비용량.xlsx", index=False, engine="openpyxl")


In [9]:
# 누적 발전량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total_2022 = pd.read_excel("dataset/지역별_누적_발전량.xlsx")
data_2024 = pd.read_excel("dataset/2024년_신규_발전량.xlsx")
data_2023 = pd.read_excel("dataset/2023년_신규_발전량.xlsx")
data_2022 = pd.read_excel("dataset/2022년_신규_발전량.xlsx")
data_2021 = pd.read_excel("dataset/2021년_신규_발전량.xlsx")

# 데이터프레임으로 변환
# gen = Power Generation = 발전량
gen_total_2022 = pd.DataFrame(data_total_2022)
gen_2024 = pd.DataFrame(data_2024)
gen_2023 = pd.DataFrame(data_2023)
gen_2022 = pd.DataFrame(data_2022)
gen_2021 = pd.DataFrame(data_2021)

# 열과 행 전치
gen_2024 = gen_2024.T.reset_index(drop=True)
gen_2023 = gen_2023.T.reset_index(drop=True)
gen_2022 = gen_2022.T.reset_index(drop=True)
gen_2021 = gen_2021.T.reset_index(drop=True)

# 컴럼 정리
gen_2024.columns = gen_2024.iloc[0]
gen_2024.drop(labels=gen_2024.index[0], axis=0, inplace=True)
gen_2023.columns = gen_2023.iloc[0]
gen_2023.drop(labels=gen_2023.index[0], axis=0, inplace=True)
gen_2022.columns = gen_2022.iloc[0]
gen_2022.drop(labels=gen_2022.index[0], axis=0, inplace=True)
gen_2021.columns = gen_2021.iloc[0]
gen_2021.drop(labels=gen_2021.index[0], axis=0, inplace=True)


# 연도 컬럼 추가
gen_2024.insert(0, "연도", np.nan)
gen_2023.insert(0, "연도", np.nan)
gen_2022.insert(0, "연도", np.nan)
gen_2021.insert(0, "연도", np.nan)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(gen_total_2022)
mds.delete_comma(gen_2024)
mds.delete_comma(gen_2023)
mds.delete_comma(gen_2022)
mds.delete_comma(gen_2021)

# 월별 데이터 연도별 데이터로 변환
gen_2024 = pd.DataFrame(gen_2024.sum()).T
gen_2023 = pd.DataFrame(gen_2023.sum()).T
gen_2022 = pd.DataFrame(gen_2022.sum()).T
gen_2021 = pd.DataFrame(gen_2021.sum()).T

# "합계" 컬럼 추가
gen_2024["합계"] = gen_2024.sum(axis=1, numeric_only=True)
gen_2023["합계"] = gen_2023.sum(axis=1, numeric_only=True)
gen_2022["합계"] = gen_2022.sum(axis=1, numeric_only=True)
gen_2021["합계"] = gen_2021.sum(axis=1, numeric_only=True)

# # 하나의 DataFrame으로 병합
gen_total = pd.concat([gen_2024, gen_2023, gen_total_2022, gen_2022, gen_2021], ignore_index=True)

# 연도 정리
gen_total = gen_total.drop(gen_total.columns[-1], axis=1)
gen_total.rename(columns={"지역": "연도"}, inplace=True)
gen_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 소수점 제거
gen_total = gen_total.astype(int)

# # 각 연도별 누적값 계산
gen_total.iloc[1, 1:] = gen_total.iloc[2, 1:] + gen_total.iloc[1, 1:] # 2023 계산
gen_total.iloc[0, 1:] = gen_total.iloc[1, 1:] + gen_total.iloc[0, 1:] # 2024 계산
gen_total.iloc[3, 1:] = gen_total.iloc[2, 1:] - gen_total.iloc[3, 1:] # 2021 계산
gen_total.iloc[4, 1:] = gen_total.iloc[3, 1:] - gen_total.iloc[4, 1:] # 2020 계산


# # 오름차순 정렬 (주석 처리 시 내림차순)
# # gen_total.sort_values(by="연도", inplace=True)
# # gen_total.reset_index(drop=True, inplace=True)

# # 2020~2024 각 연도의 지역별 누적 발전량
gen_total.head()

gen_total.to_excel("dataset/지역별_연도별_발전량.xlsx", index=False, engine="openpyxl")

In [76]:
import pandas as pd
import numpy as np
import plotly.express as px

# === 0) 데이터 정리: '연도' + 지역 컬럼들만 평균 ===
df = gen_total.copy()
if df.columns[0] != "연도":
    df = df.rename(columns={df.columns[0]: "연도"})

# '합계'가 있으면 제외
area_cols = [c for c in df.columns if c not in ["연도", "합계"]]

# 숫자형 보정
for c in area_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 연도별 지역 평균
year_mean_df = pd.DataFrame({
    "연도": df["연도"],
    "평균 발전량(MWh)": df[area_cols].mean(axis=1).round()
}).sort_values("연도")

# === 1) Plotly 막대 ===
fig = px.bar(
    year_mean_df, x="연도", y="평균 발전량(MWh)",
    title="연도별 평균 태양광 발전량(MWh)",
    text="평균 발전량(MWh)",
    template="plotly_white"
)

# 보기 좋게 라벨/축/크기
fig.update_traces(texttemplate="%{text:,.0f}", textposition="outside")
fig.update_layout(
    width=1200, height=800,
    title_x=0.5, title_y=0.96,
    title_font_size=25, title_font_color="black",
    margin=dict(l=40, r=40, t=80, b=60)
)
fig.update_yaxes(title_text="평균 발전량(MWh)", tickformat=",.0f")
fig.update_xaxes(title_text="연도")

fig.show()
# fig.write_html("year_mean_bar.html", include_plotlyjs="cdn")


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# 0) 데이터 준비: '연도' 컬럼 보정 + 2024 행 선택 + 지역 컬럼만 사용(합계 제외)
df = gen_total.copy()
if df.columns[0] != "연도":
    df = df.rename(columns={df.columns[0]: "연도"})

area_cols = [c for c in df.columns if c not in ["연도", "합계"]]
row_2024 = df.loc[df["연도"] == 2024].iloc[0]  # 첫 행이 2024가 아닐 수 있으니 안전 선택

area_2024_df = pd.DataFrame({
    "지역": area_cols,
    "발전량(MWh)": pd.to_numeric(row_2024[area_cols].values, errors="coerce")
}).sort_values("발전량(MWh)", ascending=True)

# ===== A) 세로 막대 =====
fig = px.bar(
    area_2024_df,
    x="지역", y="발전량(MWh)",
    text=area_2024_df["발전량(MWh)"].round().map(lambda v: f"{v:,.0f}"),
    color="발전량(MWh)",                 # 값에 따라 그라데이션
    color_continuous_scale="Blues",
    title="2024년도 지역별 태양광 발전량",
    template="plotly_white"
)

fig.update_layout(
    width=1200, height=800,
    title_x=0.5, title_font_size=25,
    margin=dict(l=40, r=40, t=80, b=60),
    coloraxis_colorbar=dict(title="MWh")
)
fig.update_traces(textposition="outside")
# 정렬 유지
fig.update_xaxes(title_text="지역", categoryorder="array", categoryarray=area_2024_df["지역"])
fig.update_yaxes(title_text="발전량(MWh)", tickformat=",.0f")
fig.show()


# 저장하고 싶으면:
# fig.write_html("area_2024_bar.html", include_plotlyjs="cdn")


In [74]:
import pandas as pd
import plotly.graph_objects as go

# 엑셀 로드
df = pd.read_excel("dataset/지역별_연도별_설비용량.xlsx")

# x=지역(열 이름), y=2024행(첫 행 가정) — 필요 시 df[df['연도']==2024]로 바꿔도 됨
regions = df.columns[1:18]
y = pd.to_numeric(df.iloc[0, 1:18], errors="coerce")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=regions,
    y=y,
    mode="lines+markers",
    name="2024",
    hovertemplate="<b>%{x}</b><br>설비용량: %{y:,.0f} MW<extra></extra>"
))

fig.update_layout(
    title="2024년 지역별 설비용량",
    xaxis_title="지역",
    yaxis_title="설비용량 (MW)",
    template="plotly_white"
)
fig.update_xaxes(tickangle=-45)
fig.update_yaxes(tickformat=",.0f")

fig.show()

# 파일로 저장하고 싶으면:
# fig.write_html("capacity_2024_line.html", include_plotlyjs="cdn")


In [10]:
pd.read_excel("dataset/지역별_연도별_설비용량.xlsx")

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,50,275,420,163,353,65,152,88,2012,1613,3772,6023,3842,2016,621,1943,4590,27998
1,2023,50,234,339,139,303,55,122,78,1672,1408,3072,5410,3324,1745,610,1760,4304,24625
2,2022,48,202,297,124,274,50,99,70,1459,1240,2798,4568,2886,1564,587,1612,3995,21873
3,2021,44,170,265,110,249,46,87,62,1278,1065,2522,3911,2409,1347,549,1408,3357,18879
4,2020,41,144,200,100,215,39,69,55,1056,846,2038,3054,1771,1085,425,1105,2559,14802


In [11]:
pd.read_excel("dataset/지역별_연도별_발전량.xlsx")

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,454992,1584748,1962317,910178,2054262,372182,653470,524858,9209968,8280679,20170041,34738869,19298394,10914932,3712238,11167641,28815406,154825183
1,2023,397620,1296806,1518891,747103,1694113,305236,503506,437543,7143971,6533179,15870905,27619683,14957803,8711121,3077732,8984358,23235897,123035475
2,2022,341328,1056914,1166405,614164,1386507,249449,388527,358492,5478859,4990438,12385263,21236718,11154610,6771133,2429222,6964022,17956708,94928767
3,2021,284628,854401,851754,489619,1087788,196895,282818,287876,4001713,3603271,9087437,15534479,7826973,4971554,1763312,5086129,12875255,69085910
4,2020,232973,687334,598182,375544,837087,155021,195240,224741,2768139,2462795,6310846,10918067,5318784,3562221,1189794,3488999,9022902,48348677


In [68]:
import pandas as pd

# 엑셀 불러오기
capacity = pd.read_excel("dataset/지역별_연도별_설비용량.xlsx")
generation = pd.read_excel("dataset/지역별_연도별_발전량.xlsx")

# 설비용량 long 변환 (합계 제외)
cap_long = capacity.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="설비용량(MW)"
)

# 발전량 long 변환 (합계 제외)
gen_long = generation.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="발전량(MWh)"
)

# 데이터 합치기
merged = pd.merge(cap_long, gen_long, on=["연도", "지역"])

# 설비 이용률 계산
merged["설비이용률(%)"] = merged["발전량(MWh)"] / (merged["설비용량(MW)"] * 8760) * 100

# 연도 기준 내림차순 (2024 → 2020)
merged_sorted = merged.sort_values(by="연도", ascending=False)

# 인덱스 초기화
merged_sorted = merged_sorted.reset_index(drop=True)

print(merged_sorted.head(20))  # 상위 20개 확인

merged_sorted.to_excel("dataset/합친데이터.xlsx", index=False, engine="openpyxl")


      연도  지역  설비용량(MW)  발전량(MWh)   설비이용률(%)
0   2024  서울        50     57372  13.098630
1   2024  대구       420    443426  12.052240
2   2024  광주       353    360149  11.646714
3   2024  경남      2016   2203811  12.478998
4   2024  전남      6023   7119186  13.493151
5   2024  울산       152    149964  11.262617
6   2024  인천       163    163075  11.420778
7   2024  제주       621    634506  11.663799
8   2024  경북      3842   4340591  12.896961
9   2024  충남      3772   4299136  13.010842
10  2024  강원      1943   2183283  12.827237
11  2024  세종        88     87315  11.326666
12  2024  부산       275    287942  11.952760
13  2024  전북      4590   5579509  13.876476
14  2024  충북      1613   1747500  12.367409
15  2024  경기      2012   2065997  11.721889
16  2024  대전        65     66946  11.757288
17  2023  충남      3072   3485642  12.952615
18  2023  울산       122    114979  10.758571
19  2023  세종        78     79051  11.569342


In [73]:
import numpy as np
import pandas as pd
import plotly.express as px

# ===== 0) 2024 데이터 준비 =====
df_2024 = merged_sorted[merged_sorted["연도"] == 2024].copy()
for col in ["설비용량(MW)", "발전량(MWh)", "설비이용률(%)"]:
    df_2024[col] = pd.to_numeric(df_2024[col], errors="coerce")
df_2024 = df_2024.dropna(subset=["설비용량(MW)", "발전량(MWh)", "설비이용률(%)"])

mean_cap = df_2024["설비용량(MW)"].mean()
mean_cf  = df_2024["설비이용률(%)"].mean()

# ===== 1) 사분면 구분 =====
def quadrant(row):
    cap, cf = row["설비용량(MW)"], row["설비이용률(%)"]
    if cap >= mean_cap and cf >= mean_cf:
        return "성과 우수 (1사분면)"
    elif cap < mean_cap and cf >= mean_cf:
        return "확대 잠재력 (2사분면)"
    elif cap >= mean_cap and cf < mean_cf:
        return "성능 개선 필요 (4사분면)"
    else:
        return "관심 필요 (3사분면)"

df_2024["구분"] = df_2024.apply(quadrant, axis=1)

colors = {
    "성과 우수 (1사분면)": "#2ca02c",
    "확대 잠재력 (2사분면)": "#1f77b4",
    "성능 개선 필요 (4사분면)": "#d62728",
    "관심 필요 (3사분면)": "#9467bd",
}

# ===== 2) 버블 차트 생성 (텍스트=지역, 중앙 배치) =====
fig = px.scatter(
    df_2024,
    x="설비용량(MW)",
    y="설비이용률(%)",
    size="발전량(MWh)",
    color="구분",
    text="지역",                  # 버블 안에 지역명
    hover_name="지역",
    hover_data={
        "설비용량(MW)": ":,",
        "발전량(MWh)": ":,",
        "설비이용률(%)": ":.2f",
        "구분": True
    },
    size_max=120,
    color_discrete_map=colors,
    title="2024년 지역별 설비용량 vs 설비이용률 (버블=발전량, 사분면 구분)"
)

# 평균선(사분면 경계)
fig.add_hline(y=mean_cf, line_dash="dash", annotation_text=f"평균 이용률 {mean_cf:.2f}%")
fig.add_vline(x=mean_cap, line_dash="dash", annotation_text=f"평균 용량 {mean_cap:,.0f}MW")

fig.update_layout(
    template="plotly_white",
    xaxis_title="설비용량 (MW)",
    yaxis_title="설비이용률 (%)",
    legend_title="구분",
)

# ===== 3) 실제 마커 '픽셀 크기'에 비례해 글자 크기 스케일링 =====
# - Plotly Express는 trace를 색상별로 나누므로, 각 trace의 marker.size(픽셀 단위)를 사용
# - 경험상 "지름 * 0.38" 정도가 보기 좋음. 최소/최대 글씨 크기 클리핑으로 가독성 보장.

SCALE = 0.38   # 글씨크기 = (마커지름 px) * SCALE
MIN_F = 8      # 최소 글씨 pt
MAX_F = 8     # 최대 글씨 pt

for tr in fig.data:
    # tr.marker.size: 이 trace에 해당하는 각 점의 '지름(px)'로 환산된 값 배열
    sizes_px = np.array(tr.marker.size, dtype=float)

    # 지름(px) -> 글씨 크기(pt)
    text_sizes = sizes_px * SCALE
    text_sizes = np.clip(text_sizes, MIN_F, MAX_F)

    # 가운데 배치 + 색상(흰색) 적용
    tr.textposition = "middle center"
    tr.textfont = dict(size=text_sizes.tolist(), color="white")

# (선택) 버블이 아주 작아 글씨가 안 보이면 size_max를 키우거나 MIN_F를 줄이세요.
fig.show()

fig.write_html("설비이용률.html", include_plotlyjs="cdn")


In [83]:
import re
from pathlib import Path
import webbrowser
import plotly.io as pio
from plotly.basedatatypes import BaseFigure

# === 지도(fig_map) 제외한 피겨 목록 ===
candidates = [
    ("성과/규모 버블 (2024)", "fig_bubble"),
    ("2024 지역별 발전량", "area_2024_fig"),
    ("연도별 평균 발전량", "year_mean_fig"),
    ("연도별 평균 발전량 & YoY(%)", "fig_year_yoy"),
]

def slugify(text: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "-", text).strip("-").lower()

# 현재 네임스페이스에서 피겨 수집
ns = {}
ns.update(globals())
try:
    ns.update(locals())
except Exception:
    pass

sections = []
included_titles = []
common_config = {"displaylogo": False}

for title, var_name in candidates:
    fig_obj = ns.get(var_name)
    if not isinstance(fig_obj, BaseFigure):
        continue
    div_id = slugify(title)
    snippet = pio.to_html(
        fig_obj,
        include_plotlyjs=False,      # plotly.js는 head에서 한 번만 로드
        full_html=False,
        div_id=div_id,
        config=common_config,
    )
    sections.append(f"""
    <section class="chart">
        <h2 id="{div_id}">{title}</h2>
        {snippet}
    </section>
    """)
    included_titles.append(title)

page_html = f"""<!doctype html>
<html lang="ko">
<head>
    <meta charset="utf-8"/>
    <title>재생에너지 대시보드</title>
    <meta name="viewport" content="width=device-width, initial-scale=1"/>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Noto Sans KR", Arial, "Apple SD Gothic Neo", sans-serif; margin: 0; padding: 0 80px 80px; background: #fafafa; }}
        header {{ position: sticky; top: 0; background: #ffffffcc; backdrop-filter: blur(6px); border-bottom: 1px solid #eee; }}
        .container {{ max-width: 1280px; margin: 0 auto; padding: 16px 20px; }}
        h1 {{ font-size: 20px; margin: 8px 0; }}
        nav ul {{ display: flex; gap: 12px; list-style: none; padding: 0; margin: 8px 0 0; flex-wrap: wrap; }}
        nav a {{ text-decoration: none; color: #0366d6; font-size: 14px; }}
        .chart {{ max-width: 1200px; margin: 28px auto; background: #fff; padding: 16px; border-radius: 12px; box-shadow: 0 2px 12px rgba(0,0,0,.06); }}
        .chart h2 {{ margin: 8px 6px 12px; font-size: 18px; }}
        footer {{ text-align:center; color:#777; margin-top: 40px; }}
    </style>
    </head>
    <body>
    <header>
        <div class="container">
        <h1>재생에너지 대시보드</h1>
        <nav>
            <ul>
            { "".join([f'<li><a href="#{slugify(t)}">{t}</a></li>' for t in included_titles]) }
            </ul>
        </nav>
    </div>
    </header>

    <main class="container">
        { "".join(sections) if sections else "<p>표시할 차트가 없습니다. 피겨 변수를 확인하세요.</p>" }
        <footer>© 대시보드</footer>
    </main>
</body>
</html>
"""

output_path = Path("renewable_dashboard.html").resolve()
with open(output_path, "w", encoding="utf-8") as f:
    f.write(page_html)

print(f"[OK] {output_path} 생성 완료! 브라우저로 열어보세요.")
webbrowser.open(output_path.as_uri())


[OK] C:\Users\jk316\project_renewable\project_seasonal_renewable_energy\renewable_dashboard.html 생성 완료! 브라우저로 열어보세요.


True

In [44]:
import pandas as pd
import numpy as np
from datetime import date

# === 전제: merged DataFrame 사용 (연도, 지역, 설비용량(MW), 발전량(MWh), 설비이용률(%)) ===
df = merged.copy()

# 숫자형 정리
for col in ["설비용량(MW)", "발전량(MWh)", "설비이용률(%)"]:
    df[col] = (
        df[col].astype(str).str.replace(",", "", regex=False).str.strip()
        .replace({"": np.nan, "-": np.nan})
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 설비이용률이 비어 있으면 재계산
need_cf = df["설비이용률(%)"].isna()
df.loc[need_cf, "설비이용률(%)"] = df["발전량(MWh)"] / (df["설비용량(MW)"] * 8760) * 100

# 2024만 필터 + 안전처리
df_2024 = (
    df[(df["연도"] == 2024)]
    .replace([np.inf, -np.inf], np.nan)
    .dropna(subset=["설비용량(MW)", "발전량(MWh)", "설비이용률(%)"])
)
df_2024 = df_2024[df_2024["설비용량(MW)"] > 0].copy()

# 평균값 및 2사분면 마스크
mean_cap = df_2024["설비용량(MW)"].mean()
mean_cf  = df_2024["설비이용률(%)"].mean()
mask_q2  = (df_2024["설비용량(MW)"] < mean_cap) & (df_2024["설비이용률(%)"] >= mean_cf)

cols = ["지역", "설비용량(MW)", "발전량(MWh)", "설비이용률(%)"]

# 4개 표
t1 = df_2024[cols].sort_values("지역")  # 전체
t2 = df_2024.loc[mask_q2, cols].sort_values("설비이용률(%)", ascending=False)  # 2사분면
t3 = df_2024[cols].sort_values("설비이용률(%)", ascending=False).head(10)  # 상위 10
t4 = df_2024[cols].sort_values("설비이용률(%)", ascending=True).head(10)   # 하위 10

# 표 스타일 함수
def style(df, caption):
    sty = (
        df.reset_index(drop=True)      # 인덱스 재지정(0..)
            .style
            .hide(axis="index")          # ← 인덱스 숨김 (pandas>=1.4)
            # .hide_index()              # (구버전 pandas라면 이 줄 사용)
            .format({
                "설비용량(MW)": "{:,.0f}",
                "발전량(MWh)": "{:,.0f}",
                "설비이용률(%)": "{:.2f}"
            })
            .set_table_attributes('class="tbl"')
            .set_caption(caption)
    )
    return sty.to_html()


# HTML 조립
html = f"""<!doctype html>
<html lang="ko">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>지역별 4개 표 (2024)</title>
<style>
:root {{
    --bg:#0b1020; --card:#141a2a; --text:#e8ecf1; --muted:#9aa4b2; --accent:#3b82f6; --border:#263042;
}}
body {{ margin:0; padding:24px; background:var(--bg); color:var(--text); font-family: Inter, -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; }}
h1 {{ margin:0 0 8px 0; font-size:24px; }}
.subtitle {{ color:var(--muted); margin-bottom:20px; font-size:13px; }}
.grid {{ display:grid; grid-template-columns: repeat(2, minmax(0,1fr)); gap:16px; }}
.card {{ background:var(--card); border:1px solid var(--border); border-radius:16px; padding:14px 14px 8px; box-shadow:0 6px 20px rgba(0,0,0,.25); }}
.tbl {{ width:100%; border-collapse:collapse; font-size:13px; }}
.tbl caption {{ caption-side: top; text-align:left; font-weight:600; padding:4px 0 8px; color:var(--accent); }}
.tbl th, .tbl td {{ border-bottom:1px solid var(--border); padding:8px 10px; text-align:right; }}
.tbl th:first-child, .tbl td:first-child {{ text-align:left; }}
.tbl thead th {{ position:sticky; top:0; background:#1b2336; z-index:1; }}
.badge {{ display:inline-block; background:#1b2336; color:var(--muted); padding:4px 8px; border-radius:999px; font-size:12px; margin-left:8px; }}
@media (max-width: 900px) {{ .grid {{ grid-template-columns:1fr; }} }}
</style>
</head>
<body>
    <h1>지역별 지표 표 (2024) <span class="badge">생성일: {date.today().isoformat()}</span></h1>
    <div class="subtitle">설비이용률 평균 {mean_cf:.2f}% · 설비용량 평균 {mean_cap:,.0f} MW · 2사분면(효율≥평균 & 용량&lt;평균) 강조</div>
    <div class="grid">
    <div class="card">{style(t1, "① 2024 전체 지역")}</div>
    <div class="card">{style(t2, "② 2사분면(투자유망) 지역")}</div>
    <div class="card">{style(t3, "③ 설비이용률 상위 10")}</div>
    <div class="card">{style(t4, "④ 설비이용률 하위 10")}</div>
    </div>
</body>
</html>"""

# 파일 저장
out_path = "tables_2024.html"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(html)
print(f"HTML saved -> {out_path}")

from pathlib import Path
import webbrowser

out = Path("tables_2024.html").resolve()
webbrowser.open(out.as_uri())

HTML saved -> tables_2024.html


True